### This program combines the mos_archi

In [1]:
import matplotlib
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import time
from datetime import datetime


In [41]:
def convert_to_datetime(timestamp):
    try:
        return datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
    except ValueError:
        return datetime.strptime(timestamp, "%Y-%m-%d")



In [24]:
dfmos = pd.read_csv(r'/home/daniel/projects/rails/data/mos_archive_data_hourly_data_for_all_rail_stations_sep2019_aug2023.csv', sep = ',')


In [30]:
# The data for  2021-11-04 12:00:00' are not correct in Mos Archive, so that data is removed 
df = dfmos[~(dfmos['analysis_date'] ==  '2021-11-04 12:00:00')]
df.shape

(3969240, 33)

In [22]:
def calculate_hourly_values(df, param):
    h_param = f'{param}1h'
    df_hourly_param = pd.DataFrame(columns=['lat', 'lon', 'analysis_date', 'forecast_period', h_param])
    unique_latlon_pairs = df[['lat', 'lon']].drop_duplicates()

    def process_row(row_tuple):
        index, row = row_tuple
        lat, lon = row['lat'], row['lon']
        filtered_df = df[(df['lat'] == lat) & (df['lon'] == lon)]
        analysis_dates = filtered_df['analysis_date'].unique()
        result_rows = []
        for ad in analysis_dates:
            ad_filtered_df = filtered_df[filtered_df['analysis_date'] == ad]
            period = 1
            while period <= 240:  # Adjust the condition based on your specific requirement
                #print(period)
                if period < 90:
                    increment = 1
                elif period < 144:
                    increment = 3
                else:
                    increment = 6
                period_filtered_df = ad_filtered_df[ad_filtered_df['forecast_period'] == period]
                param_values = period_filtered_df[param].values
                if len(param_values) > 0:
                    param_value = param_values[0]
                    if period == 1:
                        param_value = param_value/3600 # convert to Watts (Joules/sec)
                        result_rows.append({'lat': lat, 'lon': lon, 'analysis_date': ad, 'forecast_period': period, h_param: param_value})
                    else:
                        prev_period = period - increment
                        prev_period_filtered_df = ad_filtered_df[ad_filtered_df['forecast_period'] == prev_period]
                        prev_param_values = prev_period_filtered_df[param].values
                        if len(prev_param_values) > 0:
                            prev_param_value = prev_param_values[0]
                            param_difference = param_value - prev_param_value
                            param_value = param_difference / (increment * 3600)
                            result_rows.append({'lat': lat, 'lon': lon, 'analysis_date': ad, 'forecast_period': period, h_param: param_value})
                        else: # prev_parameter has no value
                            print(param,lat, lon, period, ad)
                else: # parameter has no value
                    print(param,lat, lon, period, ad)
                period += increment  # Increment the period using the calculated increment
        return result_rows
    result_rows = [row for rows in map(process_row, unique_latlon_pairs.iterrows()) for row in rows]
    df_hourly_param = pd.DataFrame(result_rows)

    return df_hourly_param


In [31]:
# creating the hourly values for SRR','STR', 'SLHF', 'SSHF'
import time
hourly_parameters = ['SRR','STR', 'SLHF', 'SSHF']
for param in hourly_parameters:
    print(param)
    start_time = time.time()
    df_hourly_param = calculate_hourly_values(df, param)
    df = df.merge(df_hourly_param, on=['lat', 'lon', 'analysis_date', 'forecast_period'])
    end_time = time.time()
    print(f'elapsed_time: ', end_time - start_time)

SRR
elapsed_time:  2273.999979734421
STR
elapsed_time:  2224.440554380417
SLHF
elapsed_time:  2185.547898054123
SSHF
elapsed_time:  2210.199623823166


In [32]:
df.shape

(3969240, 37)

In [33]:
df.to_csv('/home/daniel/projects/rails/data/mos_archive_data_corrected_hourly_data_for_all_rail_stations_sep2019_aug2023.csv', index=False)

In [34]:
# rail temperature data from Väylävirasto
df = pd.read_csv(r'/home/daniel/projects/rails/data/RailTemperatures_with_stations_sep2019_aug2023.csv', encoding = 'Latin-1',sep = ',')

df['lat'] = round(df['lat'],3)
df['lon'] = round(df['lon'],3)

df = df[~((df['station_id'] == 31) &  (pd.to_datetime(df['Timestamp']) >= '2022-09-01 00:00:00'))]


In [35]:
df

,station,station_id,lat,lon,X,Y,Timestamp,TAir,TRail
0,Hammaslahti,10,62.398,30.027,6922164,656442,2019-09-17 12:00:00,12.0,14.0
1,Hammaslahti,10,62.398,30.027,6922164,656442,2019-09-17 13:00:00,11.0,14.0
2,Hammaslahti,10,62.398,30.027,6922164,656442,2019-09-17 14:00:00,11.0,16.0
3,Hammaslahti,10,62.398,30.027,6922164,656442,2019-09-17 14:00:00,11.0,15.0
4,Hammaslahti,10,62.398,30.027,6922164,656442,2019-09-17 16:00:00,11.0,15.0
...,...,...,...,...,...,...,...,...,...
336897,Tupos,80,64.879,25.503,7195843,429111,2023-06-01 23:00:00,1.0,2.0
336898,Tupos,80,64.879,25.503,7195843,429111,2023-06-01 18:00:00,9.0,14.0
336899,Tupos,80,64.879,25.503,7195843,429111,2023-06-01 13:00:00,7.0,16.0
336900,Tupos,80,64.879,25.503,7195843,429111,2023-06-01 10:00:00,7.0,17.0


In [36]:
# forecast data from MOS archive
dfmos = pd.read_csv(r'/home/daniel/projects/rails/data/mos_archive_data_corrected_hourly_data_for_all_rail_stations_sep2019_aug2023.csv', sep = ',')

# rounding the latitide and longitude to 3 digits
dfmos.loc[:,'lat'] = round(dfmos['lat'],3)
dfmos.loc[:,'lon'] = round(dfmos['lon'],3)


In [37]:
# Creating  the wind speed (WS) from the U10 and V10 components
dfmos.loc[:, 'WS'] = np.sqrt(np.square(dfmos['U10']) + np.square(dfmos['V10']))


In [38]:
dfmos.columns

Index(['station_id', 'analysis_time', 'forecast_time', 'forecast_period',
       'analysis_date', 'MSL', 'T2', 'D2', 'U10', 'V10', 'LCC', 'MCC', 'SKT',
       'MX2T', 'MN2T', 'T_925', 'T2_ENSMEAN_MA1', 'SRR', 'STR', 'SLHF', 'SSHF',
       'cosmonth', 'sinmonth', 'coshour', 'sinhour', 'lat', 'lon', 'cosDoY',
       'sinDoY', 'hourly_STR', 'hourly_SLHF', 'hourly_SSHF', 'hourly_SRR',
       'SRR1h', 'STR1h', 'SLHF1h', 'SSHF1h', 'WS'],
      dtype='object')

In [42]:
# converting the dfmos forecast time in  yyyy-mm-dd format to yyyy-mm-dd HH:MM:SS format
dfmos.loc[:, 'forecast_time'] = dfmos.loc[:,'forecast_time'].apply(convert_to_datetime)
# Convert the datetime objects back to string in the desired format
dfmos.loc[:, 'forecast_time'] = dfmos.loc[:,'forecast_time'].dt.strftime('%Y-%m-%d %H:%M:%S')


/tmp/ipykernel_8096/290663528.py:2: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  dfmos.loc[:, 'forecast_time'] = dfmos.loc[:,'forecast_time'].apply(convert_to_datetime)
/tmp/ipykernel_8096/290663528.py:4: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  dfmos.loc[:, 'forecast_time'] = dfmos.loc[:,'forecast_time'].dt.strftime('%Y-%m-%d %H:%M:%S')


In [43]:
cols = ['analysis_time', 'forecast_time', 'forecast_period',
       'analysis_date', 'MSL', 'T2', 'T_925', 'D2', 'WS', 'LCC', 'MCC', 'SKT',
       'cosmonth', 'sinmonth', 'coshour', 'sinhour','cosDoY', 'sinDoY', 'lat', 'lon',
       'SRR1h', 'STR1h','SLHF1h','SSHF1h' ]


In [44]:
dfmos = dfmos[cols]

In [45]:
#  combining the rail temperature data from Väylävirasto and the ECMWF forecast data from MOS archive
dfmosrail= dfmos.merge(df,left_on = ['forecast_time','lat', 'lon'], right_on = ['Timestamp','lat', 'lon'])


In [47]:
# save the combined  data 
dfmosrail.to_csv('/home/daniel/projects/rails/data/rail_temperatures_mos_data_sep2019_aug2023_corrected_radiation_params.csv', sep = ',', index = False)